In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd
import time
from urllib.request import urlretrieve
from tqdm import tqdm
from selenium.webdriver import ActionChains
import csv

In [29]:
def driver_start(num):
    chrome_options = webdriver.ChromeOptions() 
    chrome_options.add_argument('--headless')#창없이 실행
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    #print(0)

    path='./chromedriver'
    driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
    driver.set_window_size(1024,1000)

    url = "https://www.yogiyo.co.kr/mobile/#/"
    driver.get(url)
    time.sleep(1)
    #print(1)
    driver.find_element_by_xpath('//*[@id="search"]/div/form/input').clear()
    driver.find_element_by_xpath('//*[@id="search"]/div/form/input').send_keys('서울특별시 동대문구 전농동 694 서울시립대학교')
    driver.find_element_by_xpath('//*[@id="button_search_address"]/button[2]').click()
    time.sleep(1)
    #print(2)

    driver.find_element_by_xpath(f'//*[@id="category"]/ul/li[{num}]/span').click() #치킨,피자등 카테고리 클릭
    time.sleep(1)
    #print(2)
    
    return driver

In [19]:
def scroll_element(shop,i):#음식점이 있는 곳까지 스크롤
    print(f"{i}th scroll")
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")
        try:
            some_tag = driver.find_element_by_xpath(f'//*[@title="{shop[i]}"]')
            if some_tag:
                return False
        except Exception:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height: #끝까지 스크롤후 해당 음식점이 없으면 종료 
                return True
            last_height = new_height
            time.sleep(0.5)

In [45]:
def craw(i,shop_in,driver,site_map_cnt):
    try:
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[3]').click()
    except Exception:
        driver.quit()
        driver=driver_start(site_map_cnt)
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop_in[i]}"]').click()
        time.sleep(3)
        driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/ul/li[3]').click()
    
    time.sleep(2)
    shop=driver.find_element_by_xpath('//*[@id="content"]/div[2]/div[1]/div[1]/div[1]/span').text
    notice=driver.find_element_by_xpath('//*[@id="info"]/div[1]/div[2]').text
    
    return shop, notice

In [5]:
#저장된 식당리스트 부르기
fin_list=[1,1,1,1,1,1,1,1]
with open('./data/final_shop.csv', 'r', encoding='cp949') as f:
    rdr = csv.reader(f)
    for i,line in enumerate(rdr):
        fin_list[i] = line

In [24]:
data_map={'치킨':0,'피자':1,'중식':2,'한식':3,'일식':4,'족발':5,'분식':6,'카페':7}
site_map={'치킨':5,'피자':6,'중식':7,'한식':8,'일식':9,'족발':10,'분식':12,'카페':13}

In [49]:

food_type='카페'

data_map_cnt=data_map[food_type]
site_map_cnt=site_map[food_type]
shop=fin_list[data_map_cnt]

start=0
end=len(shop)


shops=[]
notices=[]
driver.quit()
driver=driver_start(site_map_cnt)
for i in tqdm(range(start,end)):
    go_stop=scroll_element(shop,i)
    if go_stop:
        print('wrong shop')
        continue
        
    time.sleep(3)
    
    try:
        driver.find_element_by_xpath(f'//*[@title="{shop[i]}"]').click()
    except Exception:
        driver.quit()
        driver=driver_start(site_map_cnt)
        time.sleep(2)
        scroll_element(shop,i)
        time.sleep(3)
        driver.find_element_by_xpath(f'//*[@title="{shop[i]}"]').click()
    time.sleep(3)
    
    shop_name,notice=craw(i,shop,driver,site_map_cnt)
    shops.append(shop_name)
    
    print(shop_name)
    notices.append(notice)
    driver.back()
    time.sleep(2)
    
df=pd.DataFrame({'shop':shops,'notice':notices})

<ipython-input-29-77ae715e676c>:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=path,chrome_options=chrome_options)
  0%|          | 0/266 [00:00<?, ?it/s]

0th scroll
보스턴샐러드하우스


  0%|          | 1/266 [00:10<45:53, 10.39s/it]

1th scroll
빽다방-장안동사거리점


  1%|          | 2/266 [00:20<45:38, 10.37s/it]

...

 99%|█████████▉| 264/266 [1:01:03<00:32, 16.03s/it]

264th scroll
김언니샌드위치


100%|█████████▉| 265/266 [1:01:18<00:15, 15.71s/it]

265th scroll
배스킨라빈스-전농동아점


100%|██████████| 266/266 [1:01:30<00:00, 13.87s/it]


In [50]:
df.to_csv('/root/workspace/data/cafe_notice.csv')

In [47]:
df

,shop,notice
0,장안만두쟁이,■주문하시는 모든 고객님들께 콜라 500ml를\n보내드리겠습니다.■\n☆리뷰도 참여...
1,한솥도시락-휘경주공점,
2,마초떡볶이-답십리점,
3,충무할매김밥&돈까스,
4,최가네크레이지떡볶이-전농점,"""일회용 위생장갑""이 필요하시면\n""요청 사항"" 에 꼭 적어주세요.... 2-3번적..."
...,...,...
221,이경규의압구정김밥-이문점,이문동 쌍용아파트상가에 있는 압구정김밥입니다.\n신선한재료로 다양한메뉴를 합리적인 ...
222,오마뎅-장안현대홈타운점,♡리뷰이벤트 ! 유부주머니 증정♡\n주문요청란에 리뷰 이벤트 참여 작성 하시면 꼬~...
223,돈까스퐁당떡볶이공수간-답십리점,"💖떡볶이의 최강 공들인 떡볶이 ""공수간""💖\n답십리점 OPEN EVENT!! 맛있게..."
224,호텔글로리-회기본점,"리뷰이벤트-\n요청사항에 ""리뷰달께요"" 메모주시면 모듬감자튀김 추가하여 보내드립니다..."
